In [2]:
# Лабораторная работа №2
# Светлаковой Ульяны

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
file_path = '03_март_09.csv'
data = pd.read_csv(file_path, index_col=0)
data.columns = pd.to_numeric(data.columns, errors='coerce')

data.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
1,1.4,1.2,1.0,0.4,0.2,-0.3,-0.5,-0.6,-0.6,-0.5,...,9.0,-0.2,-0.3,-0.1,-0.4,NaN,-0.5,-0.3,-0.1,-0.3
2,-0.7,-1.0,-1.3,-1.7,-1.7,-1.9,-2.3,-2.6,-2.5,-2.5,...,0.8,1.7,2.3,2.4,2.2,2.1,1.6,1.4,1.5,0.9
3,0.8,1.3,1.4,1.5,2.1,2.5,2.6,2.7,2.8,2.8,...,2.5,2.7,3.4,3.4,2.8,2.5,2.4,2.3,2.3,2.2
4,2.3,2.1,1.8,1.7,1.4,1.1,0.6,0.5,0.5,0.5,...,0.9,1.4,1.7,1.7,2.1,2.3,2.2,2.1,1.9,2.4
5,2.4,2.4,1.9,1.8,1.8,1.9,1.9,1.8,1.7,1.8,...,3.4,3.9,NaN,NaN,NaN,2.4,2.4,2.2,2.2,2.2


In [5]:
def linear_interpolation(row):
    new_row = row.copy()
    n = len(new_row)
    for i in range(n):
        if np.isnan(new_row[i]):
            left, right = None, None
            for j in range(i - 1, -1, -1):
                if not np.isnan(row[j]):
                    left = (j, row[j])
                    break
            for j in range(i + 1, n):
                if not np.isnan(row[j]):
                    right = (j, row[j])
                    break
            if left and right:
                new_row[i] = left[1] + (right[1] - left[1]) * (i - left[0]) / (right[0] - left[0])
    return new_row


In [6]:
def quadratic_interpolation(row):
    new_row = row.copy() 
    n = len(new_row)
    for i in range(n):
        if np.isnan(new_row[i]):
            points = []
            for j in range(i - 1, -1, -1):
                if not np.isnan(row[j]):
                    points.append((j, row[j]))
                    if len(points) == 2:
                        break
            for j in range(i + 1, n):
                if not np.isnan(row[j]):
                    points.append((j, row[j]))
                    if len(points) == 3:
                        break
            if len(points) == 3:
                x0, y0 = points[0]
                x1, y1 = points[1]
                x2, y2 = points[2]

                A = np.array([[x0**2, x0, 1],
                              [x1**2, x1, 1],
                              [x2**2, x2, 1]])
                B = np.array([y0, y1, y2])

                a, b, c = np.linalg.solve(A, B)
                new_row[i] = a*i**2 + b*i + c
    return new_row 

In [7]:
data_copy = data.copy()
data_linear_interp = data_copy.apply(linear_interpolation, axis=1)
data_quadratic_interp = data_copy.apply(quadratic_interpolation, axis=1)

In [8]:
results = []
# Сравниваем оригинальные и интерполированные данные
for day in range(data.shape[0]):
    for hour in range(data.shape[1]):
        original_value = data.iloc[day, hour]
        linear_value = data_linear_interp.iloc[day, hour]
        quadratic_value = data_quadratic_interp.iloc[day, hour]
        
        if np.isnan(original_value) and (not np.isnan(linear_value) or not np.isnan(quadratic_value)):
            results.append({
                'День': day + 1,
                'Час': hour,
                'Оригинальное значение': original_value,
                'Линейная интерполяция': linear_value,
                'Квадратичная интерполяция': quadratic_value
            })
results_df = pd.DataFrame(results)

results_df 

,День,Час,Оригинальное значение,Линейная интерполяция,Квадратичная интерполяция
0,1,19,NaN,-0.450000,-0.533333
1,5,16,NaN,3.525000,4.050000
2,5,17,NaN,3.150000,3.850000
3,5,18,NaN,2.775000,3.300000
4,7,2,NaN,1.150000,1.166667
5,14,13,NaN,3.050000,3.066667
6,17,6,NaN,1.650000,1.666667
7,20,20,NaN,2.000000,2.066667
8,21,5,NaN,2.550000,2.533333
9,24,11,NaN,0.450000,0.300000
